In [1]:
import pyvista as pv
import numpy as np
import xarray as xr
from JulesD3D.plotPyVista import makeBottomSurface
from JulesD3D.processNetCDF import addDepth, fixCORs, fixMeshGrid, makeVelocity, addVectorSum
from JulesD3D.utils import ncFilepicker#, quickDF
from cmocean.cm import deep

In [7]:
# folder = '/Volumes/Seagate Expansion Drive/ThesisResults/Current'
folder = '/Users/julesblom/ThesisResults/MultiTest'

nc_filename = ncFilepicker(folder)

In [8]:
nc_filename

Select(description='NetCDF file:', layout=Layout(height='120px', width='100%'), options=('/Users/julesblom/The…

In [9]:
trim = xr.open_dataset(nc_filename.value) # Open with xarray : DataSet : A collection of DataArrays

In [10]:
if 'velocity' not in trim:
    trim = fixMeshGrid(trim, trim.XZ.values, trim.YZ.values, mystery_flag=True)
    trim = addDepth(trim)
    trim = makeVelocity(trim)
else:
    print("DataSet is preprocessed, 'velocity' is already in DataSet")

------ Fixing mesh grid, assuming a uniform grid ------
x_gridstep:	 200.0 m
y_gridstep:	 200.0 m
width:		 26000.0 m
length:		 36200.0 m
132 x 182 grid


In [11]:
bottom_surface_grid = makeBottomSurface(trim)

In [12]:
plot_title = f"{trim.bottom_stress.attrs['long_name']} ({trim.bottom_stress.attrs['units']})"

AttributeError: 'Dataset' object has no attribute 'bottom_stress'

In [12]:
plot_bottom_stress = trim.bottom_stress.isel(time=-5).values[:-1,:-1].T

AttributeError: 'Dataset' object has no attribute 'bottom_stress'

In [ ]:
plot_acc_sand_flux = trim.DMSEDCUM.isel(time=-2, LSEDTOT=0).values[:-1,:-1].T

In [ ]:
# grid.field_arrays['bottom_stress'] = plot_bottom_stress

In [ ]:
bottom_surface_grid.field_arrays['acc_sand_flux'] = plot_acc_sand_flux

In [ ]:
bottom_surface_grid

In [ ]:
p = pv.Plotter()
# p.add_mesh(bottom_surface_grid, scalars='bottom_stress', cmap=deep, interpolate_before_map=False)
p.add_mesh(bottom_surface_grid, scalars='acc_sand_flux', cmap='cividis', interpolate_before_map=False)
# p.add_scalar_bar(title=plot_title)
p.enable()
p.show_grid()
p.set_scale(zscale=15)
p.show()

In [ ]:
# # https://docs.pyvista.org/plotting/plotting.html#plot-time-series-data

# bottom_surface_grid.field_arrays['anim_bottom_stress'] = trim.bottom_stress.isel(time=0).values[:-1,:-1].T

# # bottom_surface_grid.set_active_scalars('bottom_stress')

# animation_plotter = pv.BackgroundPlotter()
# animation_plotter.add_mesh(grid, lighting=True, scalars='anim_bottom_stress', cmap=deep, interpolate_before_map=False)
# animation_plotter.set_scale(zscale=10)
# animation_plotter.show_grid()
# animation_plotter.show()

# def update_bottom_stress():
#     for i in range(trim.time.size):
#         if i % 10 == 0:
#             print("time", i, "max: ", trim.bottom_stress.isel(time=i).max().values)

# #         # Update scalars
#         new_bottom_stress = trim.bottom_stress.isel(time=i).values[:-1,:-1].T
#         bottom_surface_grid.field_arrays['anim_bottom_stress'] = new_bottom_stress
#         time.sleep(0.05)


        
# thread = Thread(target=update_bottom_stress)
# thread.start()

In [ ]:
# Create a plotter object and set the scalars to the Z height
plotter = pv.Plotter()
sargs = dict(height=0.25, vertical=True, position_x=0.05, position_y=0.05)
plotter.add_mesh(bottom_surface_grid, scalars=trim.bottom_stress.isel(time=0).values[:-1,:-1].T,
                 cmap=deep, interpolate_before_map=False, scalar_bar_args=sargs, stitle='Bottom Stress N/m2')
plotter.set_scale(zscale=15)
plotter.show_grid()

In [ ]:
# https://docs.pyvista.org/examples/02-plot/gif.html#sphx-glr-examples-02-plot-gif-py
# setup camera and close

plotter.show(auto_close=False)

# Open a gif
plotter.open_gif("bottomstress1.5.gif")

pts = grid.points.copy()

for i in range(trim.time.size):
    new_bottom_stress = trim.bottom_stress.isel(time=i).values[:-1,:-1].T.ravel()
#     print(trim.time.isel(time=i).values)
    plotter.update_scalars(new_bottom_stress)
    timetext = plotter.add_text("t = {}".format(trim.time.isel(time=i).values), position='upper_left')
    plotter.write_frame()
    plotter.remove_actor(timetext)
    
# Close movie and delete object
plotter.close()